In [1]:
# import packages and functions
import sys, os
sys.path.append(os.path.abspath("../src"))
import pandas as pd

from data_loader import load_price_data

C:\Users\edzz0\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
# Load raw wide-format prices
price_dir = "../data/raw_prices"
price_wide_raw = load_price_data(price_dir)

In [3]:
# Print initial universe
original_tickers = set(price_wide_raw.columns)
print(f"Initial universe: {len(original_tickers)} tickers")

Initial universe: 503 tickers


In [4]:
# Drop weekends
price_wide = price_wide_raw[price_wide_raw.index.dayofweek < 5]

In [5]:
# Drop duplicate timestamps
price_wide = price_wide[~price_wide.index.duplicated()]

In [ ]:
# Drop tickers with all prices ≤ 1

valid_price_fraction = (price_wide > 1).sum() / price_wide.notna().sum()
price_wide = price_wide.loc[:, valid_price_fraction >= 0.95]

price_wide = price_wide.loc[:, (price_wide > 1).all()]

In [11]:
import numpy as np
np.sum(price_wide['ABBV'] <= 1)

0

In [7]:
(price_wide > 1).all()

A        True
AAPL     True
ABBV    False
ABNB    False
ABT      True
        ...  
XYL     False
YUM      True
ZBH      True
ZBRA     True
ZTS     False
Length: 503, dtype: bool